In [2]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout


from google.colab import drive
drive.mount('/content/drive')

DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/Big Data/Modulo 2 Sistema de Recomendacion/"
os.chdir(DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Lee los dos archivos que contienen el listado de bebidas y los ratings dado por los usuarios
data1 = pd.read_csv("starbucks_ratings.csv", header='infer', sep=",")
data2 = pd.read_csv("starbucks_beverages.csv", header='infer')

print(data1.shape, data2.shape)

(1161, 3) (130, 3)


In [4]:
#Del primer archivo obtenemos los datos con las evaluaciones de las bebidas dadas por los usuarios

data1.head()

,userID,Beverage,rating
0,U1050,2584,5
1,U1082,2594,5
2,U1050,2594,5
3,U1082,2608,5
4,U1050,2608,5


In [5]:
#Del segundo archivo obtenemos el catalogo con el nombre de las bebidas
data2.head()

,Beverage,Beverage name,Beverage_category
0,5042,Classic Espresso Drinks-Cappuccino-2% Milk,Classic Espresso Drinks
1,5047,Classic Espresso Drinks-Cappuccino-2% Milk,Classic Espresso Drinks
2,5060,Classic Espresso Drinks-Cappuccino-2% Milk,Classic Espresso Drinks
3,5106,Classic Espresso Drinks-Cappuccino-2% Milk,Classic Espresso Drinks
4,5063,Classic Espresso Drinks-Cappuccino-Grande Nonf...,Classic Espresso Drinks


In [9]:
# Hacemos un dataframe conmbianado para tener en un mismo dataset los ratings y nombres de las bebidas

df = pd.merge(data1,data2, on='Beverage')

df.head()

,userID,Beverage,rating,Beverage name,Beverage_category
0,U1050,2584,5,Coffee-Brewed Coffee-Venti,Brewed Coffee
1,U1103,2584,1,Coffee-Brewed Coffee-Venti,Brewed Coffee
2,U1123,2584,1,Coffee-Brewed Coffee-Venti,Brewed Coffee
3,U1067,2584,2,Coffee-Brewed Coffee-Venti,Brewed Coffee
4,U1107,2584,2,Coffee-Brewed Coffee-Venti,Brewed Coffee


Implementaremos un modelo de recomendación basado en aprendizaje profundo implica el uso de técnicas avanzadas para capturar relaciones complejas entre usuarios y elementos. Una de las arquitecturas comunes para este propósito es el uso de Redes Neuronales Profundas (DNN). Aquí, vamos a construir un modelo simple de recomendación utilizando TensorFlow y Keras.

In [10]:
# Codificación de las IDs de usuarios y bebidas
user_ids = df['userID'].astype('category').cat.codes.values
item_ids = df['Beverage'].astype('category').cat.codes.values
ratings = df['rating'].values

In [11]:
# División de los datos en conjuntos de entrenamiento y prueba
X = np.array([user_ids, item_ids]).T
y = ratings
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Parámetros del modelo
n_users = df['userID'].nunique()
n_items = df['Beverage'].nunique()
embedding_size = 50



In [12]:
# Definición del modelo
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

user_embedding = Embedding(input_dim=n_users, output_dim=embedding_size, name='user_embedding')(user_input)
item_embedding = Embedding(input_dim=n_items, output_dim=embedding_size, name='item_embedding')(item_input)

user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)

concat = Concatenate()([user_vec, item_vec])

dense = Dense(128, activation='relu')(concat)
dropout = Dropout(0.5)(dense)
output = Dense(1)(dropout)

model = Model([user_input, item_input], output)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

model.summary()

# Entrenamiento del modelo
history = model.fit([X_train[:, 0], X_train[:, 1]], y_train,
                    validation_data=([X_test[:, 0], X_test[:, 1]], y_test),
                    epochs=20, batch_size=32, verbose=1)

# Evaluación del modelo
loss, mae = model.evaluate([X_test[:, 0], X_test[:, 1]], y_test, verbose=1)
print(f"Loss: {loss}")
print(f"Mean Absolute Error: {mae}")

# Ejemplo de predicción para un usuario y una bebida específicos
user_id = 0  # ID codificado del usuario 'U1050'
item_id = 0  # ID codificado de la bebida '2584'
pred_rating = model.predict([np.array([user_id]), np.array([item_id])])
print(f"Predicción de la calificación para el usuario {user_id} y la bebida {item_id}: {pred_rating[0][0]}")

# Generar recomendaciones para un usuario específico
def recommend_items(user_id, model, n_items, top_n=5):
    user_ids = np.array([user_id] * n_items)
    item_ids = np.arange(n_items)
    predictions = model.predict([user_ids, item_ids])
    top_items = np.argsort(predictions.flatten())[-top_n:][::-1]
    return top_items

recommended_items = recommend_items(user_id, model, n_items, top_n=3)
print(f"Top-3 recomendaciones para el usuario {user_id}: {recommended_items}")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 item_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 50)                6900      ['user_input[0][0]']          
                                                                                                  
 item_embedding (Embedding)  (None, 1, 50)                6500      ['item_input[0][0]']          
                                                                                              